In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Import required libraries 

import json 
import pandas as pd 
import nltk
import random , math

In [4]:
# Importing tokenized nutritionix Dataset

res_health = pd.read_csv("trainingdataset_tokenized.csv")
res_health.head()
res_health.shape
res_health['healthy'].value_counts()

,Unnamed: 0,fields.item_name,fields.brand_name,healthy,clean_item_name,filtered_item_name
0,0,Chili Cheese Fries,Taco Bell Canada,0,"['chili', 'cheese', 'fries']","['chili', 'chees', 'fri']"
1,1,Steak Chalupa Supreme,Taco Bell Canada,0,"['steak', 'chalupa', 'supreme']","['steak', 'chalupa', 'suprem']"
2,2,7-Layer Burrito - Chicken,Taco Bell Canada,0,"['7layer', 'burrito', 'chicken']","['7layer', 'burrito', 'chicken']"
3,3,Fresco Beef Crunchy Taco,Taco Bell Canada,1,"['fresco', 'beef', 'crunchy', 'taco']","['fresco', 'beef', 'crunchi', 'taco']"
4,4,Cinnamon Twists,Taco Bell Canada,1,"['cinnamon', 'twists']","['cinnamon', 'twist']"


(3050, 6)

0    1798
1    1252
Name: healthy, dtype: int64

In [5]:
# Splitting dataset into training and test

from sklearn.model_selection import train_test_split

feature_data = res_health[['filtered_item_name','healthy']]
feature_data.head()

random.seed(100)
#random.shuffle(feature_data)


x_train ,x_test = train_test_split(feature_data,test_size=0.5)
x_test['healthy'].value_counts()
x_train['healthy'].value_counts()

,filtered_item_name,healthy
0,"['chili', 'chees', 'fri']",0
1,"['steak', 'chalupa', 'suprem']",0
2,"['7layer', 'burrito', 'chicken']",0
3,"['fresco', 'beef', 'crunchi', 'taco']",1
4,"['cinnamon', 'twist']",1


0    901
1    624
Name: healthy, dtype: int64

0    897
1    628
Name: healthy, dtype: int64

In [10]:
# Vectorizing Training Dataset and applying vectrization on training and test dataset

from sklearn.feature_extraction.text import CountVectorizer


# Perform vectrization
vector = CountVectorizer()
vector.fit(x_train["filtered_item_name"])
vector = CountVectorizer(vocabulary=vector.vocabulary_) #vocabulary is a parameter, it should be vocabulary_ as it is an attribute.


# Apply vectrization

training_data = vector.transform(x_train["filtered_item_name"])
test_data = vector.transform(x_test["filtered_item_name"])


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [11]:
# Building Naive Bayes classifier 

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn import metrics


# Fitting classifier on Training data
model = MultinomialNB().fit(training_data, x_train['healthy'])

# Predicting labels for Test data
test_predict = model.predict(test_data)

# Checking accuracy of model on Test data
confusion_matrix(x_test['healthy'],test_predict)
metrics.accuracy_score(x_test['healthy'],test_predict)

array([[767, 134],
       [257, 367]], dtype=int64)

0.7436065573770492

In [12]:
# Loading the yelp dataset

yelp = pd.read_csv("yelp_final_tokenized.csv")
yelp_test = pd.DataFrame(yelp['filtered_caption'])
yelp_test.head()
yelp_test.shape

,filtered_caption
0,"['12oz', 'boneless', 'ribey', 'grill', 'perfec..."
1,"['warm', 'chocol', 'browni', 'melt', 'caramel'..."
2,"['yummi', 'tart', 'chantilli', 'guava', 'salt'..."
3,"['tart', 'chocol', 'haupia', 'ube', 'guava', '..."
4,"['spam', 'musubi']"


(58237, 1)

In [8]:
# Performing tokenization on yelp dataset and generating perdictions by applying naive bayes classfier

# Applying tokenization
yelp_test_data = vector.transform(yelp_test["filtered_caption"])

# Predicting with Naive Bayes classifier
yelp_predict = model.predict(yelp_test_data)

# Storing predictions
yelp["prediction_score"] = yelp_predict


In [9]:
# Generating final yelp dataset with predicted flags and average health scores for each business

yelp_final = yelp.groupby(['business_id','name', 'address','city','state','postal_code','latitude','longitude','stars','review_count','categories'])[['prediction_score']].mean()
yelp_final['healthy_percent'] = pd.Series(["{0:.2f}%".format(val * 100) for val in yelp_final['prediction_score']], index = yelp_final.index)
yelp_final


# Exporting yelp predictions to CSV
yelp_final.to_csv('Yelp_Healthy_restaurant_predictions.csv',  header=True)




,,,,,,,,,,,prediction_score,healthy_percent
business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,,
--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.841694,-79.399755,3.0,55,"Chinese, Restaurants",0.333333,33.33%
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.123183,-115.169190,4.0,1759,"Seafood, Cajun/Creole, Restaurants, Steakhouses",0.250000,25.00%
--S62v0QgkqQaVUhFnNHrw,Denny's,6207 Wilson Mills Rd,Highland Heights,OH,44143,41.539420,-81.455225,2.0,56,"American (Traditional), Breakfast & Brunch, Diners, Restaurants",0.000000,0.00%
--SrzpvFLwP_YFwB_Cetow,Keung Kee Restaurant,"3300 Midland Avenue, Unit 41",Toronto,ON,M1V 0C7,43.806750,-79.288858,3.5,43,"Restaurants, Chinese",0.333333,33.33%
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,1408 E Blvd,Charlotte,NC,28203,35.199776,-80.844439,4.0,389,"Restaurants, Spanish, Peruvian, Latin American, Colombian",0.000000,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...
zyPGYeXF4XKCqNN1pjFWhg,La Locanda,"6830 E 5th Ave, Ste 108",Scottsdale,AZ,85251,33.495795,-111.933013,4.5,225,"Italian, Bars, Wine Bars, Cocktail Bars, Seafood, Nightlife, Restaurants",0.000000,0.00%
zyw5DjrRks7a8OhmBsgCQQ,The Pour House Pub And Kitchen,182 Dupont Street,Toronto,ON,M5R 2E6,43.675708,-79.403853,3.0,114,"Nightlife, Canadian (New), Pubs, Restaurants, Irish, Bars",0.000000,0.00%
zzdJkVrJ8OkZPUxrvLMkYQ,Saint Sushi Westmount,1359 Avenue Greene,Westmount,QC,H3Z 2A9,45.487640,-73.590782,3.5,16,"Japanese, Sushi Bars, Restaurants",0.000000,0.00%
